<a href="https://colab.research.google.com/github/mich3leRusso/Deep_learning_last/blob/main/u_net_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample submission based on 3D U-Net as a baseline

In this notebook, we'll go through a full attempt at a solution - albeit a poorly performing one.

**Full disclosure:** this notebook was not developed on Kaggle, but on another machine. It may not work out of the box but hopefully, it can serve as a useful starting point for now.

The idea is to train a 3D U-Net to segment the bugs based on the single bug images. Here, there is only one bug per image, so we can use the 3D U-Net to segment the bug. Since we know the label of the bugs in the single bug images, we can also train the U-Net to predict the bug type, by casting it as a multi-class segmentation problem (13 classes, 12 bugs + background). We can then use the segmentation to find its center point and use that for our submission.

In [ ]:
import os
import glob

import monai
import numpy as np
import torch
from tqdm import tqdm, trange

In [ ]:
# Path to individual bugs (change to where you store the bugs)
data_dir = 'path/to/train'

First, we gather all the files and their labels to prepare the data loaders.
We will store the labels as one-hot encoded vectors.

In [ ]:
images = []
labels = []

class_names = os.listdir(data_dir)
class_image_files = [
    glob.glob(os.path.join(data_dir, name, '*')) for name in class_names
]

for i, files in enumerate(class_image_files):
    images += files

    labels_i = torch.zeros(len(files), len(class_names))
    labels_i[:, i] = 1
    labels.append(labels_i)


labels = torch.cat(labels, dim=0)

Some samples of the lists we made:

In [ ]:
class_names

In [ ]:
images[:5]

In [ ]:
labels[:5]

Looks good! Let's split the individual images into a train and validation set:

In [ ]:
from sklearn.model_selection import train_test_split

train_val_split = 0.6

train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, train_size=train_val_split, stratify=labels,
    random_state=1337,
)

During the training pipeline, the rough idea is this:

1. Load a batch of images and their labels.
2. Segment the (single) bug using a threshold on the image intensity.
3. Use the label to turn the binary thresholded image into an actual segmentation mask.
4. Train the U-Net to predict the segmentation mask.

The solution here is based on the MONAI library, which provides building blocks for 3D image segmentations. Most of the above steps can be achieved via existing MONAI transforms we can add as data augmentations. However, we need two extra helper classes.

First, a class to tell it how to read our 3D tiff-files:

In [ ]:
from monai.data import ImageReader
from skimage.io import imread

class TiffReader(ImageReader):
    def __init__(self):
        super().__init__()


    def get_data(self, img):
        return np.asarray(img), dict()


    def read(self, data, **kwargs):
        if isinstance(data, str):
            data = [data]

        return [imread(f) for f in data]


    def verify_suffix(self, filename):
        return filename.endswith('.tif') or filename.endswith('.tiff')


Next a class to copy an image

In [ ]:
from copy import deepcopy
from monai.transforms import MapTransform

class CopyImaged(MapTransform):
    def __init__(self, key_to_copy, new_key, allow_missing_keys=False):
        super().__init__(key_to_copy, allow_missing_keys)
        self.key_to_copy = key_to_copy
        self.new_key = new_key


    def __call__(self, data):
        data = dict(data)
        data[self.new_key] = deepcopy(data[self.key_to_copy])
        return data


We can now set up our transforms:

In [ ]:
import monai.transforms as T
from skimage.io import imread

# Precompute padding to get equal side lengths
im0 = imread(images[0])
shape = torch.tensor(im0.shape)
padding = (0, int((shape[0] - shape[1]) // 2), int((shape[0] - shape[2]) // 2))

# Prepare train and test data loaders
keys = ['image', 'mask']
train_transforms = T.Compose([
    # Load the image
    T.LoadImaged(keys='image', reader=TiffReader, image_only=True),
    T.Resized(keys='image', spatial_size=shape),
    # Scale the intensity
    T.ScaleIntensityd(keys='image'),
    # Pad the image so all sides are equal
    T.BorderPadd(keys='image', spatial_border=padding),
    # Make a copy of the image, which is what will later be our mask
    CopyImaged(key_to_copy='image', new_key='mask'),
    # Smooth the image and then threshold it
    T.GaussianSmoothd(keys='mask', sigma=2),
    T.AsDiscreted(keys='mask', threshold=0.25, dtype=torch.long),
    # Only keep the largest connected component as the bug mask
    T.KeepLargestConnectedComponentd(keys='mask', applied_labels=[0]),
    # Apply normal data augmentations
    T.EnsureTyped(keys=['image', 'mask', 'label'], track_meta=False),
    T.RandAffined(
        keys=['image', 'mask'], prob=0.95, rotate_range=(np.pi/2,) * 3,
        translate_range=shape // torch.tensor([2, 1, 1]), padding_mode='zeros'),
    T.RandAxisFlipd(keys=keys, prob=0.5),
    T.RandScaleIntensityd(keys='image', factors=0.25, prob=0.5),
    T.RandZoomd(keys=keys, prob=0.5),
    T.SqueezeDimd(keys='mask'),
    T.CastToTyped(keys='mask', dtype=torch.long),
])

val_transforms = T.Compose([
    # See train_transforms
    T.LoadImaged(keys='image', reader=TiffReader, image_only=True),
    T.Resized(keys='image', spatial_size=shape),
    T.ScaleIntensityd(keys='image'),
    T.BorderPadd(keys='image', spatial_border=padding),
    CopyImaged(key_to_copy='image', new_key='mask'),
    T.GaussianSmoothd(keys='mask', sigma=2),
    T.AsDiscreted(keys='mask', threshold=0.25, dtype=torch.long),
    T.KeepLargestConnectedComponentd(keys='mask', applied_labels=[0]),
    T.EnsureTyped(keys=['image', 'mask', 'label'], track_meta=False),
    T.SqueezeDimd(keys='mask'),
    T.CastToTyped(keys='mask', dtype=torch.long),
])

Let's quickly run through how this looks like when the code runs.

We create our `Dataset`s

In [ ]:
from monai.data import Dataset

train_data = Dataset(
    [{ 'image': f, 'label': l }
        for f, l in zip(train_images, train_labels)],
    transform=train_transforms,
)

val_data = Dataset(
    [{ 'image': f, 'label': l }
     for f, l in zip(val_images, val_labels)],
    transform=val_transforms,
)

...and `DataLoaders`

In [ ]:
from monai.data import DataLoader

train_batch_size = 4
val_batch_size = 2

train_loader = DataLoader(
    train_data,
    shuffle=True,
    num_workers=0,
    batch_size=train_batch_size,
)
val_loader = DataLoader(
    val_data,
    num_workers=0,
    shuffle=False,
    batch_size=val_batch_size,
)

Get a batch and look at it:

In [ ]:
for batch in train_loader:
    break

In [ ]:
batch.keys()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 4, figsize=(10, 5))
for i in range(4):
    ax[0, i].imshow(batch['image'][i, 0].cpu().numpy().max(axis=0), cmap='gray')
    ax[0, i].axis('off')
    ax[1, i].imshow(batch['mask'][i].cpu().numpy().max(axis=0), cmap='tab20b', vmin=0, vmax=1)
    ax[1, i].axis('off')
fig.tight_layout()

In [ ]:
mask_label = batch['mask'] * (batch['label'].argmax(dim=1).view(-1, 1, 1, 1) + 1)

fig, ax = plt.subplots(2, 4, figsize=(10, 5))
for i in range(4):
    ax[0, i].imshow(batch['image'][i, 0].cpu().numpy().max(axis=0), cmap='gray')
    ax[0, i].axis('off')
    ax[1, i].imshow(mask_label[i].cpu().numpy().max(axis=0), cmap='tab20b',
                    vmin=0, vmax=len(class_names), interpolation='none')
    ax[1, i].axis('off')
fig.tight_layout()

Alright, now for real:

In [ ]:
train_batch_size = 4
val_batch_size = 8

train_loader = DataLoader(
    train_data,
    shuffle=True,
    num_workers=8,
    batch_size=train_batch_size,
    drop_last=True,
)
val_loader = DataLoader(
    val_data,
    num_workers=8,
    shuffle=False,
    batch_size=val_batch_size,
    drop_last=True,
)

We use the built in U-Net implementation from MONAI:

In [ ]:
model = monai.networks.nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=len(class_names) + 1,  # Bug classes and background
    channels=(32, 64, 128, 256),
    strides=(2, 2, 2),
    num_res_units=2,
)

In [ ]:
model = model.cuda()

We use a mix of cross entropy and Dice loss. For the cross entropy loss, we weigh class 0 (background) with 0.1, since there's a lot of it.

In [ ]:
ce_weight = torch.ones(len(class_names) + 1, device='cuda')
ce_weight[0] = 0.1

loss_fn = monai.losses.DiceCELoss(
    ce_weight=ce_weight,
    to_onehot_y=True,
    softmax=True,
    include_background=True,
)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4,
)

Run the training. This will take several hours sadly...

In [ ]:
val_every = 1

train_losses = []
val_losses = []
epoch_train_losses = []
epoch_val_losses = []
val_loss_mean = None
best_metric = float('inf')
for epoch in range(100):
    model.train()

    loss_mean = 0
    num_batches = 0
    for batch in tqdm(train_loader):
        inputs = batch['image'].cuda()
        label = batch['label'].argmax(dim=1).cuda()
        expected = batch['mask'].cuda() * (label.view(-1, 1, 1, 1) + 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, expected.unsqueeze(1))

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            loss_mean += loss.item()
            num_batches += 1
            train_losses.append(loss.item())


    loss_mean /= num_batches
    epoch_train_losses.append(loss_mean)

    if epoch % val_every == 0:
        model.eval()

        val_loss_mean = 0
        num_batches = 0
        for batch in tqdm(val_loader):
            inputs = batch['image'].cuda()
            label = batch['label'].argmax(dim=1).cuda()
            expected = batch['mask'].cuda() * (label.view(-1, 1, 1, 1) + 1)
            with torch.no_grad():
                outputs = model(inputs)
                loss = loss_fn(outputs, expected.unsqueeze(1))
                val_loss_mean += loss.item()
                num_batches += 1
                val_losses.append(loss.item())

        val_loss_mean /= num_batches
        epoch_val_losses.append(val_loss_mean)

        if val_loss_mean <= best_metric:
            print('Saving new best model')
            best_metric = val_loss_mean
            torch.save(model.state_dict(), 'best_model.pth')

    checkpoint = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'epoch_train_losses': epoch_train_losses,
        'epoch_val_losses': epoch_val_losses,
        'best_metric': best_metric,
    }
    torch.save(checkpoint, 'latest_model.ckpt')

    print(f"Epoch {epoch + 1}, loss: {loss_mean:.4f}, val. loss: {val_loss_mean:.4f}")

In [ ]:
epoch

Let's see the val losses

In [ ]:
plt.plot(epoch_val_losses)

Not much and then we suddenly drop to good performance. Weird, but we'll take it for now.

It would likely be smart to look further into this - but for now, let's just use the trained model to predict the test set.

## Predictions on single bugs

First, we'll try it out on the validation examples with single bugs, just as a sanity check that it works for the trained task. To do this, we'll make a helper function:

In [ ]:
from skimage import measure

def predict(image):
    with torch.no_grad():
        pred = model(image)
    pred_sm = pred.softmax(dim=1).cpu().numpy()
    class_props = []
    for i in range(1, 13):
        props = measure.regionprops(
            measure.label(pred_sm[0, i] > 0.5)
        )
        props = [p for p in props if p.area > 5**3]
        class_props.append(props)

    boxes = np.array([p.bbox for props in class_props for p in props])
    centers = np.array([p.centroid for props in class_props for p in props])
    labels = np.array([i + 1 for i, props in enumerate(class_props) for p in props])
    scores = np.ones(len(boxes))
    return {
        'boxes': boxes,
        'centers': centers,
        'labels': labels,
        'scores': scores,
        'pred': pred_sm,
    }

Now, load an image and run a prediction:

In [ ]:
from skimage.io import imread

fname = val_images[70]
image = imread(fname)
print(fname)

In [ ]:
image = np.pad(image / 255.0, ((0, 0), (32, 32), (32, 32)))
pred = predict(torch.tensor(image[None, None]).float().cuda())

In [ ]:
np.unique(pred['pred'][0].argmax(axis=0))

In [ ]:
from matplotlib.patches import Rectangle

a = 1

#fig, ax = plt.subplots(figsize=(10, 10))
fig, ax = plt.subplots()
ax.imshow(image.max(axis=a), cmap='gray')
ax.imshow(pred['pred'][0].argmax(axis=0).max(axis=a), alpha=0.5, interpolation='none')
for box, center, label in zip(pred['boxes'], pred['centers'], pred['labels']):
    if a == 0:
        center = (center[2] - 0.5, center[1] - 0.5)
        xy = (box[2] - 0.5, box[1] - 0.5)
        height = box[4] - box[1]
        width = box[5] - box[2]
    elif a == 1:
        center = (center[2] - 0.5, center[0] - 0.5)
        xy = (box[2] - 0.5, box[0] - 0.5)
        height = box[3] - box[0]
        width = box[5] - box[2]
    else:
        center = (center[1] - 0.5, center[0] - 0.5)
        xy = (box[1] - 0.5, box[0] - 0.5)
        height = box[3] - box[0]
        width = box[4] - box[1]
    ax.add_patch(
        Rectangle(xy, width, height, edgecolor='r', facecolor='none'))
    ax.plot(center[0], center[1], 'ro')
    ax.text(xy[0], xy[1] - 1, class_names[label - 1], color='r')

# Predictions on mixed volumes

Satisfied it works on the single bugs, we'll now try the validation examples with a mixture of bugs:

In [ ]:
mix_images = sorted(glob.glob('path/to/validation/*.tif'))

In [ ]:
fname = mix_images[10]
image = imread(fname)
image = np.pad(image / 255.0, ((0, 0), (18, 18), (18, 18)))  # Pad to even side length
image.shape

In [ ]:
plt.imshow(image.max(axis=1), cmap='gray')

In [ ]:
pred = predict(torch.tensor(image[None, None]).float().cuda())

In [ ]:
np.unique(pred['pred'][0].argmax(axis=0))

In [ ]:
from matplotlib.patches import Rectangle

a = 2

#fig, ax = plt.subplots(figsize=(10, 10))
fig, ax = plt.subplots()
ax.imshow(image.max(axis=a), cmap='gray')
ax.imshow(pred['pred'][0].argmax(axis=0).max(axis=a), alpha=0.5, interpolation='none')
for box, center, label in zip(pred['boxes'], pred['centers'], pred['labels']):
    if a == 0:
        center = (center[2] - 0.5, center[1] - 0.5)
        xy = (box[2] - 0.5, box[1] - 0.5)
        height = box[4] - box[1]
        width = box[5] - box[2]
    elif a == 1:
        center = (center[2] - 0.5, center[0] - 0.5)
        xy = (box[2] - 0.5, box[0] - 0.5)
        height = box[3] - box[0]
        width = box[5] - box[2]
    else:
        center = (center[1] - 0.5, center[0] - 0.5)
        xy = (box[1] - 0.5, box[0] - 0.5)
        height = box[3] - box[0]
        width = box[4] - box[1]
    ax.add_patch(
        Rectangle(xy, width, height, edgecolor='r', facecolor='none'))
    ax.plot(center[0], center[1], 'ro')
    ax.text(xy[0], xy[1] - 1, class_names[label - 1], color='r')

Performance is not *great*, but it's finding something. This is to be expected when the model has only seen single bugs during training. Someone should get on that ;-)

Let's now try to format our predictions in the competition format. First, we load the sample CSV file

In [ ]:
import pandas as pd

validation = pd.read_csv('path/to/validation/validation.csv')

In [ ]:
validation.set_index('filename', inplace=True)
validation

Now, let's load the row corresponding to our current file

In [ ]:
from pathlib import Path

fname = Path(fname)
fname.name

and unpack it to get the points

In [ ]:
val_string = validation.loc[fname.name][0]
true_labels = val_string.split(';')[::4]
true_centers = np.array([
    [float(x) for x in val_string.split(';')[1::4]],
    [float(x) for x in val_string.split(';')[2::4]],
    [float(x) for x in val_string.split(';')[3::4]],
]).T

true_centers = true_centers[:, [2, 1, 0]]
true_centers += np.array([[0, 18, 18]])

print(true_labels)
print(true_centers)

Let's now plot them. Importantly, we use the same plotting code as before, so we can be fairly confident that our predictions have the correct coordinates

In [ ]:
a = 1

#fig, ax = plt.subplots(figsize=(10, 10))
fig, ax = plt.subplots()
ax.imshow(image.max(axis=a), cmap='gray')
for center, label in zip(true_centers, true_labels):
    if a == 0:
        center = (center[2] - 0.5, center[1] - 0.5)
    elif a == 1:
        center = (center[2] - 0.5, center[0] - 0.5)
    else:
        center = (center[1] - 0.5, center[0] - 0.5)
    ax.plot(center[0], center[1], 'ro')
    ax.text(center[0], center[1] - 1, label, color='r')

Looks good. Let's now make a helper function that takes a list of points and packs into the format needed for a competition solution:

In [ ]:
def pack_prediction(centers, labels):
    centers = centers - np.array([[0, 18, 18]])
    strings = []
    for l, c in zip(labels, centers):
        strings.append(f"{l};{c[2]};{c[1]};{c[0]}")
    return ";".join(strings)

Testing it out:

In [ ]:
validation.loc[fname.name][0]

In [ ]:
pack_prediction(true_centers, true_labels)

Looks good. It packs the data back to the same format.

Now, we need to convert our prediction labels to the string labels used in the competition:

In [ ]:
pred_text_labels = [class_names[l - 1].lower() for l in pred['labels']]
pred_text_labels

and we can now build a competition submission row for this image:

In [ ]:
pack_prediction(pred['centers'], pred_text_labels)

Let's now run a prediction run on all the validation volumes

In [ ]:
submission = {
    'filenames': [],
    'centerpoints': [],
}
for fname in tqdm(mix_images):
    image = imread(fname)
    image = np.pad(image / 255.0, ((0, 0), (18, 18), (18, 18)))
    pred = predict(torch.tensor(image[None, None]).float().cuda())
    pred_text_labels = [class_names[l - 1].lower() for l in pred['labels']]
    pred_text = pack_prediction(pred['centers'], pred_text_labels)
    submission['filenames'].append(Path(fname).name)
    submission['centerpoints'].append(pred_text)

Our submission would then be:

In [ ]:
submission = pd.DataFrame(submission)
submission

Which matches the format from the supplied csv file:

In [ ]:
validation

# Predictions on public test volumes

Finally, we can run a prediction run for the actual test volumes to build an actual submission:

In [ ]:
mix_images = sorted(glob.glob('path/to/test/*.tif'))

In [ ]:
submission = {
    'filename': [],
    'centerpoints': [],
}
for fname in tqdm(mix_images):
    image = imread(fname)
    image = np.pad(image / 255.0, ((0, 0), (18, 18), (18, 18)))
    pred = predict(torch.tensor(image[None, None]).float().cuda())
    pred_text_labels = [class_names[l - 1].lower() for l in pred['labels']]
    pred_text = pack_prediction(pred['centers'], pred_text_labels)
    submission['filename'].append(Path(fname).name)
    submission['centerpoints'].append(pred_text)

In [ ]:
submission = pd.DataFrame(submission)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

That's it - ready to submit!